In [2]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
dir_path = './' #local
# dir_path = './drive/MyDrive/BAKA/' #colab

In [7]:
#uncomment in colab
# from google.colab import drive
# drive.mount('/content/drive')
# pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', sep=' ', index_col=False )
# test = np.genfromtxt(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', delimiter=' ',dtype='float64')
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', header=None, sep=' ')
df = df.drop(columns=[31])
sp_df = np.array_split(df, 2)
sp_df[0]

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,152.028,174.270,114.872,135.012,115.821,169.357,108.251,86.5737,86.4014,60.6628,...,19.15310,37.6444,296.277,5.277580,5.69833,303.563,-4.53744,7.62196,12.33870,35.1608
1,106.969,174.270,138.093,122.560,112.017,151.941,139.409,67.5126,93.4567,141.7770,...,0.97681,54.9755,295.547,-61.058000,8.98806,317.394,-4.13179,85.07790,5.64757,43.2827
2,115.931,174.270,114.917,134.286,175.119,127.521,177.996,58.0159,69.8662,178.0840,...,-2.39748,82.3194,336.737,-44.982000,13.94950,337.090,-7.44972,63.64330,21.32290,29.9727
3,129.774,90.000,151.379,164.207,147.675,133.193,153.299,53.3455,64.2682,55.6463,...,6.15326,45.3346,348.796,1.379250,19.70390,370.834,-9.76106,19.74900,17.69820,71.4496
4,168.765,174.270,156.177,141.628,146.640,128.245,177.981,80.4555,66.8833,71.7721,...,7.91760,45.8886,362.047,13.636300,27.27890,388.642,-8.96836,15.10440,19.16930,97.3733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,155.025,173.721,137.026,141.175,142.413,144.205,171.451,71.6357,76.7435,77.4298,...,-5.88642,50.2178,384.405,1.723980,18.24680,398.022,-10.44550,15.77150,9.59649,110.1400
96,156.901,173.395,136.497,140.909,141.700,144.869,170.816,71.1895,76.4634,77.2505,...,-6.63851,50.7962,385.537,0.938872,18.37180,398.392,-10.57410,15.82180,9.31662,109.5280
97,158.522,173.625,136.929,141.031,141.552,145.678,170.873,71.2914,76.3473,76.4080,...,-7.09323,50.9274,386.211,0.622400,18.48320,398.925,-10.66650,15.72430,9.22443,109.0020
98,158.859,174.598,137.177,140.760,141.265,147.142,171.260,71.2260,76.3613,76.1615,...,-7.47433,51.2412,386.845,0.361023,18.55280,399.310,-10.78070,15.59450,8.98942,109.1230


In [9]:
files = os.listdir(dir_path + 'DataCollection')
files

['1', '2', '3', '4', '5', '6']

In [10]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
#         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(columns=[31])
        sp_df = np.array_split(df, 2)
        for p in sp_df:
            x_test.append(p.to_numpy())
            y_test.append(int(i)-1);
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(columns=[31])
        sp_df = np.array_split(df, 2)
        for p in sp_df:
            x_train.append(p.to_numpy())
            y_train.append(int(i)-1);
    
    print(len(samples), ' ', num_tests)
    
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)

301   60
256   51
151   30
205   41
211   42
151   30


In [11]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)

print(x_train)

x_train.shape:  (2042, 100, 31)
y_train.shiape:  (2042,)
x_test.shape:  (508, 100, 31)
y_test.shape:  (508,)
[[[146.102   174.27    146.022   ...  12.747     8.94849 116.923  ]
  [138.963   176.016   145.252   ...  12.831     9.10288 116.791  ]
  [133.62    177.874   145.35    ...  13.1807    9.07881 117.362  ]
  ...
  [159.741   177.26    140.089   ...  12.1147    7.51379 113.455  ]
  [159.704   176.964   140.108   ...  12.1307    7.5447  113.331  ]
  [159.641   176.734   140.127   ...  12.1575    7.58026 113.334  ]]

 [[159.564   176.371   140.2     ...  12.1792    7.62375 113.332  ]
  [159.478   176.007   140.309   ...  12.1938    7.66359 113.347  ]
  [159.383   175.67    140.465   ...  12.2283    7.69677 113.433  ]
  ...
  [154.893   172.515   141.491   ...  12.4594    7.48152 112.73   ]
  [154.995   173.132   141.052   ...  12.4302    7.43961 112.92   ]
  [155.21    173.614   140.785   ...  12.4084    7.43464 112.993  ]]

 [[155.597   173.91    140.623   ...  12.3902    7.44884 11

In [10]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

In [11]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [12]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (1021, 200, 31)
y_train.shiape:  (1021,)
x_test.shape:  (254, 200, 31)
y_test.shape:  (254,)


In [13]:
model = Sequential()
model.add(LSTM(units=100, input_shape=x_train.shape[1:], return_sequences=True,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(30, activation='softmax'))

model.add(Dense(6, activation='softmax',dtype='float64'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 200)          185600    
_________________________________________________________________
batch_normalization (BatchNo (None, 200, 200)          800       
_________________________________________________________________
dropout (Dropout)            (None, 200, 200)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 200, 200)          320800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 200, 200)          800       
_________________________________________________________________
dropout_1 (Dropout)          (None, 200, 200)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 200)          3

In [14]:
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-5)

checkpoint_filepath = 'Checkpoints'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=800,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=80,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

Epoch 1/800
12/12 [==============================] - 48s 1s/step - loss: 1.3952 - accuracy: 0.4987 - val_loss: 1.7343 - val_accuracy: 0.3786
Epoch 2/800
12/12 [==============================] - 11s 953ms/step - loss: 0.2330 - accuracy: 0.9267 - val_loss: 1.6937 - val_accuracy: 0.3786
Epoch 3/800
12/12 [==============================] - 11s 958ms/step - loss: 0.1376 - accuracy: 0.9457 - val_loss: 1.6599 - val_accuracy: 0.3398
Epoch 4/800
12/12 [==============================] - 11s 957ms/step - loss: 0.1135 - accuracy: 0.9653 - val_loss: 1.6251 - val_accuracy: 0.3981
Epoch 5/800
12/12 [==============================] - 11s 956ms/step - loss: 0.0982 - accuracy: 0.9694 - val_loss: 1.5850 - val_accuracy: 0.4272
Epoch 6/800
12/12 [==============================] - 11s 959ms/step - loss: 0.0784 - accuracy: 0.9734 - val_loss: 1.5426 - val_accuracy: 0.3981
Epoch 7/800
12/12 [==============================] - 12s 962ms/step - loss: 0.0590 - accuracy: 0.9739 - val_loss: 1.4921 - val_accuracy: 0.

KeyboardInterrupt: ignored

In [ ]:
# model.save(dir_path + 'Models/gestures.h5')

In [15]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=80)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_custom', save_format='tf')

Evaluate on test data
4/4 [==============================] - 1s 300ms/step - loss: 0.1531 - accuracy: 0.9567
test loss, test acc: [0.15312451124191284, 0.9566929340362549]


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_custom/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_custom/assets


In [ ]:
# new_model = load_model(dir_path + 'Models/gestures.h5')
# print("Evaluate on test data")
# results = new_model.evaluate(x_test, y_test, batch_size=80)
# print("test loss, test acc:", results)

In [17]:
min_max_scaler.data_min_

array([ 9.30245e+01,  8.99999e+01,  9.00456e+01,  9.00455e+01,
        9.00455e+01,  6.54812e+01,  2.57840e+01,  2.77705e+01,
        2.79286e+01,  2.04904e+01, -4.24721e+02,  6.97324e+01,
       -2.97270e+02, -4.20951e+02,  5.95510e+01, -3.15328e+02,
       -4.48325e+02,  6.33254e+01, -3.17749e+02, -4.46258e+02,
        8.67906e+01, -3.13954e+02, -4.57886e+02,  8.58241e+01,
       -3.02162e+02, -4.42757e+02,  1.03148e+02, -2.82183e+02,
        1.86895e-01,  6.48663e-02,  1.07620e-01])